In [178]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch.nn as nn
import torchtext

In [34]:
# Directories of the dataset 
train_original_directory = os.path.join("Data","Original_data","train_essays.csv")
test_original_directory = os.path.join("Data","Original_data","test_essays.csv")
train_original_prompts_directory = os.path.join("Data","Original_data","train_prompts.csv")

In [35]:
train_original_df = pd.read_csv(train_original_directory)
test_original_df = pd.read_csv(test_original_directory)
train_original_prompts_df = pd.read_csv(train_original_prompts_directory)

# Exploratory Data Analysis

In [36]:
# generated -> 0 means written by humans 
train_original_df.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [37]:
# Data Cleaning should be independent from the dataLoader (Usefull if we plan to use machine learning model as well
# Embedding should be injected into the dataloader class (Makes embedding indepent from the dataloading part 
# At the moment we don't care about the prompt text but might be usefull in future processing 

In [38]:
class CreateDataset:
    """
    This class is just to clean the dataset and the output of this class should be a cleaned dataset
    """
    def __init__(self,values:list = None):
        self.__paths : list[str] = []
        
        if values:
            self.__paths = [*values]
        
    @property
    def paths(self):
        return self.paths
    
    @paths.setter
    def paths(self,value):
        self.paths.append(value)
        
    def clean(self):
        final_df = None
        for path in self.paths:
            temp_df = pd.read_csv(path)
            if(path.split('/') == "Original"):
                temp_df = self.cleanOriginal(temp_df)
            
            if final_df is None:
                final_df = temp_df
            else:
                final_df = pd.concat([final_df,temp_df])
            
    def cleanOriginal(self,temp_df):
        # TODO: Drop the promptId and Id
        return temp_df

In [39]:
temp = CreateDataset()

In [40]:
train_final_df = train_original_df[["text","generated"]]

In [41]:
train_final_df

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [77]:
model_name = "microsoft/deberta-v3-xsmall"

tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [83]:
len(tokenizer.get_vocab())

128001

In [78]:
tokenizer(train_final_df.iloc[0]["text"])["input_ids"]

[1,
 11673,
 260,
 11673,
 286,
 331,
 441,
 515,
 306,
 1181,
 2167,
 267,
 262,
 11537,
 268,
 261,
 335,
 4166,
 3692,
 994,
 263,
 1119,
 262,
 362,
 4848,
 1193,
 260,
 11673,
 286,
 1313,
 266,
 852,
 985,
 267,
 316,
 469,
 406,
 1131,
 515,
 393,
 260,
 420,
 394,
 261,
 355,
 281,
 1392,
 264,
 900,
 337,
 9781,
 640,
 4119,
 338,
 282,
 266,
 397,
 576,
 260,
 502,
 351,
 261,
 9781,
 262,
 380,
 265,
 2020,
 520,
 282,
 266,
 397,
 576,
 264,
 333,
 260,
 344,
 334,
 912,
 265,
 291,
 261,
 1030,
 261,
 307,
 2514,
 2324,
 97689,
 261,
 1982,
 23905,
 589,
 4031,
 11673,
 261,
 309,
 293,
 4859,
 44077,
 1603,
 261,
 361,
 19835,
 281,
 262,
 107550,
 265,
 13550,
 261,
 399,
 1686,
 938,
 1549,
 292,
 814,
 9458,
 289,
 2460,
 2670,
 264,
 365,
 308,
 2002,
 260,
 11632,
 504,
 361,
 291,
 269,
 266,
 1288,
 48368,
 264,
 763,
 1827,
 264,
 1684,
 10326,
 1698,
 5685,
 292,
 97646,
 260,
 28223,
 2020,
 281,
 1744,
 270,
 621,
 864,
 265,
 10326,
 1698,
 5685,
 267,
 1611,


In [79]:
len(tokenizer(train_final_df.iloc[1]["text"])["input_ids"])

547

In [80]:
len(train_final_df.iloc[0]["text"])

3289

In [46]:
len(train_final_df.iloc[1]["text"])

2738

In [143]:
tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

0

In [215]:
# Create a DataLoader class 
class DetectionDataset(Dataset):
    
    def __init__(self,df,Tokenizer = None,train = True,max_length = 100):
        self.tokenizer = Tokenizer
        self.train = train
        self.df = df
        self.max_length = max_length
        self.padded_token = 0 if self.tokenizer is None else self.tokenizer.convert_tokens_to_ids(self.tokenizer.pad_token)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, item):
        """
        :return: text containing the indexes in numpy array / list
        """
        if self.train:
            if self.tokenizer is None: 
                return {"text" : self.df.iloc[item]["text"],"score" : self.df.iloc[item]["generated"]}
            else:
                vectorized_text = self.vectorize(self.df.iloc[item]["text"])
                return {"text" : vectorized_text,"score" : self.df.iloc[item]["generated"]}
        else:
            pass
        
    def pad(self,vector,length):
        result = np.ones(length) * self.padded_token
        result[:len(vector)] = vector
        return result
    
    def collate(self,batch):
        max_length = max([len(item['text']) for item in batch])
        texts = [self.pad(item['text'],max_length) for item in batch]
        scores = [item['score'] for item in batch]    
        return {'text': torch.LongTensor(texts), 
                'score': torch.tensor(scores, dtype=torch.float32)}
        
    
    def vectorize(self,text):
        return self.tokenizer(text)["input_ids"]

In [216]:
detectionDataset = DetectionDataset(train_final_df,tokenizer)

In [217]:
train_dataloader = DataLoader(detectionDataset, batch_size=32, shuffle=True, collate_fn=detectionDataset.collate)

In [218]:
len(tokenizer.get_vocab())

128001

In [238]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(128001,1000)
    def forward(self,x):
        x = self.embedding(x) # B x T X C here B is batch size  T is time step size and 
        return x

In [252]:
from typing import Optional

# class SinusoidalPositionalEmbedding(nn.Embedding):
#     """This module produces sinusoidal positional embeddings of any length."""
#     def __init__(self, num_positions: int, embedding_dim: int, padding_idx: Optional[int] = None) -> None:
#         super().__init__(num_positions, embedding_dim)
#         self.weight = self._init_weight(self.weight)
# 
#     @staticmethod
#     def _init_weight(out: nn.Parameter) -> nn.Parameter:
#         """
#         Interleaved sine and cosine position embeddings
#         """
#         out.requires_grad = False
#         out.detach_()
# 
#         N, D = out.shape
# 
#         ## TODO: Create a N x D//2 array of position encodings (argument to the sine/cosine)
#         inds = np.arange(0, D // 2)
#         k = np.arange(N)
#         denom = 1 / np.power(10_000, 2*inds / D)
#         position_enc = np.outer(k, denom)  # Efficiently make N x D//2 array for all positions/dimensions
#         #####
# 
#         out[:, 0::2] = torch.FloatTensor(np.sin(position_enc))  # Even indices get sin
#         out[:, 1::2] = torch.FloatTensor(np.cos(position_enc))  # Odd indices get cos
#         return out
# 
#     @torch.no_grad()
#     def forward(self, input_ids_shape: torch.Size, past_key_values_length: int = 0) -> torch.Tensor:
#         """`input_ids_shape` is expected to be [bsz x seqlen]."""
#         bsz, seq_len = input_ids_shape[:2]
#         positions = torch.arange(
#             past_key_values_length, past_key_values_length + seq_len, dtype=torch.long, device=self.weight.device
#         )
#         return super().forward(positions)
    
class AntHillTransformer(nn.Module):
    def __init__(self, d_model=128, n_head=4, n_layers=3):
        super().__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.n_layers = n_layers

        # Input embedding layers
        # TODO: Create input embedding and position encoding layers
        self.input_embeddings = nn.Embedding(num_embeddings=128001, embedding_dim=d_model, padding_idx=0)
        # self.position_encoding = SinusoidalPositionalEmbedding(num_positions=32, embedding_dim=d_model)
        ###
        self.input_layer_norm = nn.LayerNorm(d_model)

        # Transformer layers
        # TODO: Create encoder layer and transformer encoder stack
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.d_model,
            nhead=self.n_head,
            dim_feedforward=2*d_model,
            dropout=0.1,
            activation="gelu",
            batch_first=True,
            norm_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=self.n_layers)
        ###

        # Output layers
        self.output_norm = nn.LayerNorm(d_model)  # With pre-layer norm transformer, we should do one more before output
        # TODO: Create output linear layer
        self.output_layer = nn.Linear(128, 1)
        ###

    def forward(self, input_ids, padding_mask):
        """
        Parameters:
        -----------
        inputs: torch.LongTensor of shape (B, N, D)
        padding_mask: torch.BoolTensor of shape (B, N)
        
        Returns:
        --------
        outputs: torch.FlaotTensor of shape (B, N)
        """
        # TODO: Fill in forward layer
        # Inputs
        embedding_vectors = self.input_embeddings(input_ids)
        # position_vectors = self.position_encoding(input_ids.shape)
        inputs = embedding_vectors   # Add position encodings
        inputs = self.input_layer_norm(inputs)

        # Transformer
        outputs = self.transformer_encoder(inputs)

        # Output layers
        outputs = self.output_norm(outputs)
        outputs = self.output_layer(outputs)
        outputs = outputs.squeeze()
        ###
        return outputs
    

In [253]:
model = AntHillTransformer()

/opt/miniconda3/envs/kaggle/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [254]:
# model = Transformer()

In [255]:
temp = next(iter(train_dataloader))

In [256]:
result = model(temp["text"],0)

In [257]:
result.shape

torch.Size([32, 991])

In [237]:
temp["text"].shape

torch.Size([32, 1085])